In [53]:
import pandas as pd
%matplotlib inline
data = pd.read_csv('./data/exam/CM2.csv', na_values='?')
data = data.dropna(axis=0)

In [43]:
from sklearn import preprocessing
def bin_feature(df, feature):
    binarizer = preprocessing.LabelBinarizer()
    nw = binarizer.fit_transform(df[feature])
    classes = binarizer.classes_
    new_names = list(map(lambda cls: feature + "==" + str(cls), classes))
    if len(binarizer.classes_) == 2:
        new_names = new_names[1:]
    new_data = pd.DataFrame(nw, columns=new_names)
    new_data.index = df.index
    new_data = pd.concat([df, new_data], axis=1)
    new_data = new_data.drop(feature, axis=1)
    return new_data

In [44]:
from sklearn import cross_validation
data = bin_feature(data, 'C1')
data = bin_feature(data, 'C2')
X = data.drop(['MPG'], axis=1)
Y = data['MPG']
x_train, x_test, y_train, y_test = cross_validation.train_test_split(X, Y, test_size=0.2, random_state=11)

In [52]:
from sklearn import ensemble
import math
best_tree = 10
best_score = 1000000000.
for tree_count in range(10, 500, 70):
    model = ensemble.RandomForestRegressor(n_estimators=tree_count, max_depth=8, random_state=11)
    model.fit(x_train, y_train)
    res = model.predict(x_test)
    fail = 0.
    for (r, w) in zip(res, y_test):
        fail += (r - w) * (r - w)     
    fail = math.sqrt(fail)
    if fail < best_score:
        best_tree = tree_count
        best_score = fail

print('Best result: trees = {}, fail = {}'. format(best_tree, best_score))

Best result: trees = 430, fail = 22.29896789028649
